<a href="https://colab.research.google.com/github/NavneetSajwan/VAE/blob/main/Implementing_variational_autoencoders_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader

import numpy as np

## Data preparation and loading

In [ ]:
transform = transforms.Compose(
    [
        transforms.PILToTensor(),
        transforms.ConvertImageDtype(torch.float),
        transforms.Normalize(0.5, 0.5)
        ]
    )

In [ ]:
train_set = MNIST("data", train = True, transform= transform, download= True)
val_set = MNIST("data", train= False, transform = transform, download = False)

100%|██████████| 9912422/9912422 [00:00<00:00, 105247238.16it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 21996675.84it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 33947523.83it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 8340861.98it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
#create data loader
trainloader = DataLoader(dataset = train_set,
                         batch_size = 64,
                         drop_last=True,
                         shuffle = True)
val_loader = DataLoader(dataset = val_set,
                         batch_size = 64,
                         drop_last=True,
                         shuffle = True)
trainloader

In [ ]:
trainiter = iter(trainloader)
train_batch = next(trainiter)
train_batch[0].shape, train_batch[1].shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [ ]:
train_batch[0].view(64, 1, -1).shape

torch.Size([64, 1, 784])

## Model definition

- It inherits nn.Module
- It has two methods:
  - __init__:
    - Initialize the constructor from base class: `super(*class_name*, self).__init__()`
    - Formula for size of activations after convolutions:
      - `output = (input - kernel + 2* padding)/stride +1`

  - forward
    - Reshape the inputs if required.


In [ ]:
class VAE(nn.Module):
  def __init__(self, latent_dim=20):
    super(VAE, self).__init__()

    #encode layers
    self.fc1 = nn.Linear(in_features = 28 * 28, out_features = 400) # 784 > 400
    self.fclogvar = nn.Linear(in_features = 400, out_features = latent_dim) # 400 > 20
    self.fcmu = nn.Linear(in_features = 400, out_features = latent_dim)# 400 > 20

    #decode layers
    self.fc2 = nn.Linear(in_features = latent_dim, out_features = 400) # 20 > 400
    self.fc3 = nn.Linear(in_features = 400, out_features = 28*28)# 400 > 784

    self.relu = nn.ReLU()
    # self.dropout = nn.Dropout(0.5)


  def encode(self, x):
    x = self.relu(self.fc1(x)) # [bs, 400]
    _logvar = self.fclogvar(x) # [bs, 20]
    _mu = self.fcmu(x) # [bs, 20]
    return _logvar, _mu

  def reparameterize(self, _logvar, _mu):
    epsilon = torch.randn_like(_logvar)
    std = torch.exp(0.5*_logvar) # use std instead of logvar
    x = _mu + std * epsilon
    return x

  def decode(self, x):
    x = self.relu(self.fc2(x)) # [bs, 400]
    x = self.fc3(x) # [bs, 400]
    return F.sigmoid(x) #note: no activation function before sigmoid

  def forward(self, x):
    x = x.view(-1, 28 * 28)
    # print(x.shape)
    logvar, mu = self.encode(x)
    x = self.reparameterize(logvar, mu)
    x = self.decode(x)
    return x, logvar, mu

In [ ]:
# Loss function: Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, logvar, mu):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='mean')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE, KLD

## Training

In [ ]:
from torch.optim import Adam
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [ ]:
class Train:
  def __init__(self, loader, criterion, epochs):
    self.epochs = epochs
    self.plot = True
    self.log = True
    self.val = True
    self.log_lrs, self.losses = 0., 0.
    self.train_loss_list,  self.val_loss_list= [], []
    self.device = "cuda" if torch.cuda.is_available() else "cpu"
    self.model = VAE(latent_dim=20).to(self.device)
    self.trainloader, self.val_loader = loader[0], loader[1]
    self.opt = Adam(self.model.parameters(), lr= 5e-5)
    self.criterion = criterion

  def plot_lrs(self):
    plt.plot(self.log_lrs, self.losses)
    plt.xlabel("Log Learning Rate")
    plt.ylabel("Loss")
    plt.show()

  def plot_losses(self):
    # Plot Training Loss
    plt.figure(figsize=(12, 6))
    plt.plot(self.train_loss_list, label="Training Loss", color='blue')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training Loss over Epochs")
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

    if self.val:
      # Plot Testing Loss
      plt.figure(figsize=(8, 4))
      plt.plot(self.val_loss_list, label="Testing Loss", color='red')
      plt.xlabel("Epochs")
      plt.ylabel("Loss")
      plt.title("Testing Loss over Epochs")
      plt.legend()
      plt.grid(True, which='both', linestyle='--', linewidth=0.5)
      plt.show()


  def get_val_loss(self):
    self.model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    with torch.no_grad():
      for images, labels in self.val_loader:
          images, labels = images.to(self.device), labels.to(self.device)
          output, logvar, mu = self.model(images)
          bce, kld = self.criterion(output, images, logvar, mu)
          loss = bce + kld
          test_loss += loss.item()
    return test_loss / len(self.val_loader)

  def find_lr(self, init_value=1e-8, final_value=10.0, beta=0.98):
      num = len(self.trainloader) - 1  # total number of batches
      mult = (final_value / init_value) ** (1 / num)  # rate at which the lr is increased at each step
      lr = init_value
      self.opt.param_groups[0]['lr'] = lr  # setting the initial learning rate
      avg_loss = 0.
      best_loss = 0.
      batch_num = 0
      losses = []
      log_lrs = []
      trainloader_bar = tqdm(self.trainloader, str(len(self.trainloader)))
      for images, labels in trainloader_bar:
          batch_num += 1
          # get the loss for this mini-batch of inputs/outputs
          self.opt.zero_grad()
          images, labels = images.to(self.device), labels.to(self.device)
          outputs, logvar, mu = self.model(images)
          bce, kld = self.criterion(outputs, images, logvar, mu)
          loss = bce
          # compute the smoothed loss
          avg_loss = beta * avg_loss + (1 - beta) * loss.item()
          smoothed_loss = avg_loss / (1 - beta**batch_num)
          # stop if the loss is exploding
          if batch_num > 1 and smoothed_loss > 4 * best_loss:
              self.log_lrs, self.losses = log_lrs, losses
              print(f"best loss: {best_loss}")
              return "loss> 4 * best loss"
          # record the best loss
          if smoothed_loss < best_loss or batch_num == 1:
              best_loss = smoothed_loss
          # store the values
          losses.append(smoothed_loss)
          log_lrs.append(np.log10(lr))
          # do the SGD step
          loss.backward()
          self.opt.step()
          # update the lr for the next step
          lr *= mult
          self.opt.param_groups[0]['lr'] = lr
      self.log_lrs, self.losses = log_lrs, losses

  def __call__(self, lr):
    self.opt.param_groups[0]['lr'] = lr
    for epoch in range(self.epochs):
      self.model.train()
      epoch_train_loss = 0.0
      for i, (images, labels) in tqdm(enumerate(self.trainloader), str(len(self.trainloader))):
        #load data to gpu
        images.requires_grad
        images, labels = images.to(self.device), labels.to(self.device)

        #forward pass
        output, logvar, mu = self.model(images)

        #get loss
        bce, kld = self.criterion(output, images, logvar, mu)
        loss = bce

        #loss collection over epoch
        if self.log:
          epoch_train_loss += loss.item()

        # backward pass
        self.opt.zero_grad()
        loss.backward()
        self.opt.step()

      if self.log:
        avg_train_loss = epoch_train_loss / len(self.trainloader)
        self.train_loss_list.append(avg_train_loss)
        print(f"Epoch {epoch} : Train Loss : {avg_train_loss}")
        if self.val:
          avg_val_loss = self.get_val_loss()
          print(f"Epoch {epoch} : Val Loss : {avg_val_loss}")
          self.val_loss_list.append(avg_val_loss)

In [ ]:
#notes: bce loss is way too high compared to kld loss and doesn't converge below a cwertain value.
# maybe latent vector is way too small

# can't plot kld loss

In [ ]:
#initialize trainloop
learner = Train([trainloader, val_loader], loss_function, 5)

In [ ]:
learner.find_lr(init_value = 1e-5,final_value = 100)

In [ ]:
learner.plot_lrs()

In [ ]:
#start training
learner(1e-4)

937: 0it [00:00, ?it/s]

Epoch 0 : Train Loss : -69.52233739546723
Epoch 0 : Val Loss : 90604298187.48718


937: 0it [00:00, ?it/s]

Epoch 1 : Train Loss : -70.4683937634577
Epoch 1 : Val Loss : 12536596783944.205


937: 0it [00:00, ?it/s]

Epoch 2 : Train Loss : -69.13128642974822
Epoch 2 : Val Loss : 121649568088274.05


937: 0it [00:00, ?it/s]

Epoch 3 : Train Loss : -68.82087442078483
Epoch 3 : Val Loss : 170319188621994.66


937: 0it [00:00, ?it/s]

Epoch 4 : Train Loss : -68.25125470970585
Epoch 4 : Val Loss : 83804359306883.28


In [ ]:
trainloop.plot_losses()

## Testing

## Inference

In [ ]:
image, _ = train_set[0]

In [ ]:
output, _, _ = trainloop.model(image.cuda())
output = output.view(-1,28,28)

In [ ]:
topilimage = transforms.ToPILImage()
topilimage(output)